In [1]:
import pandas as pd

In [2]:
raw_df = pd.read_csv('../../data/cars.csv')

We dont what the features (0 to 9) are for. Hence dropping these features.

In [3]:
df = raw_df.drop(columns=['feature_0','feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7','feature_8','feature_9'], axis=1)
df.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,universal,False,owned,all,10900.00,False,Минская обл.,9,13,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,universal,False,owned,all,5000.00,True,Минская обл.,12,54,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,suv,False,owned,all,2800.00,True,Минская обл.,4,72,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,sedan,False,owned,all,9999.00,True,Минская обл.,9,42,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,universal,False,owned,all,2134.11,True,Гомельская обл.,14,7,7


In [4]:
df.isnull().sum()

manufacturer_name     0
model_name            0
transmission          0
color                 0
odometer_value        0
year_produced         0
engine_fuel           0
engine_has_gas        0
engine_type           0
engine_capacity      10
body_type             0
has_warranty          0
state                 0
drivetrain            0
price_usd             0
is_exchangeable       0
location_region       0
number_of_photos      0
up_counter            0
duration_listed       0
dtype: int64

There are 10 null values in engine_capacity, so we can drop them.

In [5]:
df = df.dropna()
df.isnull().sum()

manufacturer_name    0
model_name           0
transmission         0
color                0
odometer_value       0
year_produced        0
engine_fuel          0
engine_has_gas       0
engine_type          0
engine_capacity      0
body_type            0
has_warranty         0
state                0
drivetrain           0
price_usd            0
is_exchangeable      0
location_region      0
number_of_photos     0
up_counter           0
duration_listed      0
dtype: int64

Ensure prices are represented as floats

In [6]:
df.dtypes

manufacturer_name     object
model_name            object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_has_gas          bool
engine_type           object
engine_capacity      float64
body_type             object
has_warranty            bool
state                 object
drivetrain            object
price_usd            float64
is_exchangeable         bool
location_region       object
number_of_photos       int64
up_counter             int64
duration_listed        int64
dtype: object

In [7]:
df['price_usd'] = df['price_usd'].astype('float64')

There's about 45 different car manufacturing companies with different model names. To keep the dimensionality manageable, we can bin the car ranges according to affordability based off the price of the cars. 

In [8]:
df['price_usd'] = df['price_usd'].astype('float64')
temp = df.copy()

table = temp.groupby(['manufacturer_name'])['price_usd'].mean()
temp = temp.merge(table.reset_index(), how='left', on='manufacturer_name')
bins = [0,10000,25000,50000]
cars_bins = ['Budget','Medium', 'Highend']

In [9]:
df['CarRange'] = pd.cut(temp['price_usd_y'], bins,  right=False, labels=cars_bins)
df.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,duration_listed,CarRange
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,...,False,owned,all,10900.00,False,Минская обл.,9,13,16,Budget
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,...,False,owned,all,5000.00,True,Минская обл.,12,54,83,Budget
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,...,False,owned,all,2800.00,True,Минская обл.,4,72,151,Budget
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,...,False,owned,all,9999.00,True,Минская обл.,9,42,86,Budget
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,...,False,owned,all,2134.11,True,Гомельская обл.,14,7,7,Budget


We can leave out variables like "manufacturer_name","model_name","location region" now that we have efficiently binned the data into groups.

In [10]:
df.drop(['manufacturer_name','model_name','location_region'], axis=1, inplace=True)

It would be a good idea to do some scaling across the dataframe to prepare for training.

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
num_vars= ['odometer_value', 'engine_capacity', 'number_of_photos','up_counter','duration_listed', 'price_usd']
df[num_vars] = scaler.fit_transform(df[num_vars])

One-hot encode the categorical features.

In [12]:
# Define a function to generate dummy variables and merging it with data frame

def dummies(x,df):
    temp = pd.get_dummies(df[[x]], drop_first=True)
    df = pd.concat([df,temp], axis=1)
    df.drop([x], axis=1, inplace=True)
    return df

# Apply function to the df df
df = dummies('transmission', df)
df = dummies('color', df)
df = dummies('engine_fuel', df)
df = dummies('engine_has_gas', df)
df = dummies('engine_type', df)
df = dummies('body_type', df)
df = dummies('has_warranty', df)
df = dummies('state', df)
df = dummies('drivetrain', df)
df = dummies('is_exchangeable', df)
df = dummies('CarRange', df)

In [13]:
df

,odometer_value,year_produced,engine_capacity,price_usd,number_of_photos,up_counter,duration_listed,transmission_mechanical,color_blue,color_brown,...,body_type_sedan,body_type_suv,body_type_universal,body_type_van,state_new,state_owned,drivetrain_front,drivetrain_rear,CarRange_Medium,CarRange_Highend
0,0.190000,2010,0.294872,0.217984,0.094118,0.006452,0.007168,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,0.290000,2002,0.358974,0.099982,0.129412,0.028495,0.037186,0,1,0,...,0,0,1,0,0,1,0,0,0,0
2,0.402000,2001,0.294872,0.055981,0.035294,0.038172,0.067652,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,0.010000,1999,0.358974,0.199964,0.094118,0.022043,0.038530,1,1,0,...,1,0,0,0,0,1,0,0,0,0
4,0.280000,2001,0.294872,0.042663,0.152941,0.003226,0.003136,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,0.290000,2000,0.423077,0.054981,0.047059,0.045161,0.134857,0,0,0,...,1,0,0,0,0,1,1,0,0,0
38527,0.321000,2004,0.256410,0.095982,0.035294,0.010215,0.142025,1,1,0,...,0,0,0,0,0,1,1,0,0,0
38528,0.777957,2000,0.423077,0.085982,0.023529,0.033333,0.165323,0,1,0,...,1,0,0,0,0,1,1,0,0,0
38529,0.020000,2001,0.230769,0.079982,0.070588,0.083333,0.219534,1,0,0,...,0,0,0,0,0,1,1,0,0,0
